In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import pandas as pd

import json

In [ ]:
# Spark Constants
APP_NAME = 'assignment1'
MASTER = 'local[*]'

# Input Constants
INPUT_METADATA_FILE = 'tracks.csv'
INPUT_FEATURES_FILE = 'features.csv'

# Data Columns
COLUMN_TRACK_ID = 'track_id'
COLUMN_SUBSET = 'set_subset'

# Application Constants
SUBSET_SMALL_VALUE = 'small'

In [ ]:
conf = SparkConf().setAppName(APP_NAME).setMaster(MASTER)
sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession.builder.appName(APP_NAME).master(MASTER).getOrCreate()

In [ ]:
tracks = pd.read_csv(INPUT_METADATA_FILE, index_col=0, header=[0, 1])
features = pd.read_csv(INPUT_FEATURES_FILE, index_col=0, header=[0, 1, 2])

In [ ]:
small = tracks[tracks['set', 'subset'] == 'small']
small.shape

In [ ]:
ids = set(small.index.to_list())
features_sm = features.loc[ids]
features_sm.shape